### Minor Changes for feedback Loop: 


Helpful link explaining some of the nuances of this problem: https://www.reddit.com/r/adventofcode/comments/e7aqcb/2019_day_7_part_2_confused_with_the_question/

Main things to include:

- All 5 amps now need an internal list that keeps track of where they are....class would be nice but not doing it this far in. 

- We only feed the phase combination (5-9) the first iteration, after that it is all based on inputs from other Amplifiers following A -> B -> C -> D -> E -> A -> B.....

- weirdly need to pass permut_input for A, then 0, then start using output from E.....

In [21]:
def intcode_pc(list_vals, init_val, permut_input, input_sig, input_cnt):
    
    """Built in Day 5"""
    
    while True:
    
        # identify the opt_code_str
        opt_code_str = str(list_vals[init_val])

        # take actual opt code: 
        opt_code = int(opt_code_str[-2:])

        # modes can be 0 == position or 1 == immediate 
        # handle mode with try / except
        mode_3, mode_2, mode_1 = 0, 0, 0

        # overwrite 0s UNLESS type error, then we know we had trailing 0s
        # saw example here: https://p.szy.io/MrmgEA/python
        try:
            mode_1 = int(opt_code_str[-3])
            mode_2 = int(opt_code_str[-4])
            mode_3 = int(opt_code_str[-5])

        except IndexError:
            pass


        # handling each type of opt code
        if opt_code == 1:

            # the third param provides the writing point: we always look at the 3rd value after opt code to write
            write_ix = list_vals[init_val + 3]

            # we now need values to sum
            # When Mode == 0: first move to param, and take val. l_v[init+n] - output
            # Then find the list_vals[output] and this is the value of interest
            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]
            ans = val1 + val2

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 2:

            write_ix = list_vals[init_val + 3]

            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]
            ans = val1 * val2

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 3:
            # DAY 7 MOD: no more input, just pass in vals
            write_ix = list_vals[init_val + 1]
            
            if input_cnt > 0:
                list_vals[write_ix] = input_sig
                input_cnt += 1
                
            else:
                list_vals[write_ix] = permut_input
                input_cnt += 1
                
            init_val += 2


        elif opt_code == 4:
            # DAY 7: MOD to not print, but return at last line
            print_val = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            init_val += 2
            return print_val, init_val, list_vals

        elif opt_code == 5:

            # jump-if-true
            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]

            if val1 != 0:
                init_val = val2
            else:
                init_val += 3

        elif opt_code == 6:

            # jump-if-false
            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]

            if val1 == 0:
                init_val = val2
            else:
                init_val += 3

        elif opt_code == 7:

            # less than
            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]

            write_ix = list_vals[init_val + 3]

            ans = 1 if val1 < val2 else 0
            list_vals[write_ix] = ans
            init_val += 4

        elif opt_code == 8:

            # equals
            val1 = list_vals[list_vals[init_val + 1]] if mode_1 == 0 else list_vals[init_val + 1]
            val2 = list_vals[list_vals[init_val + 2]] if mode_2 == 0 else list_vals[init_val + 2]

            write_ix = list_vals[init_val + 3]

            ans = 1 if val1 == val2 else 0
            list_vals[write_ix] = ans
            init_val += 4

        elif opt_code == 99:
            flag_99 = True
            
            # need to pause?
            
            return None, init_val, list_vals

        else:

            print("Something is broken!")

#### actually building this out: 

In [22]:
import itertools
all_combinations = list(itertools.permutations([5,6,7,8,9]))

In [23]:
test = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

In [24]:
input_cnt = 0 # initialize

In [25]:
output, ix, vals = intcode_pc(test, 0, 9, 0, 0)

In [17]:
# add to dict
mem_dict = {}
mem_dict[amp_type] = (ix, vals)

In [40]:
# have a 0 start val for final answer
final_answer = 0

# iterate through just test that works - ensure output matches 
comb = (9,8,7,6,5)
counter = 0

test_case = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

# start all at index 0
mem_dict['a'] = (0, test_case.copy())
mem_dict['b'] = (0, test_case.copy())
mem_dict['c'] = (0, test_case.copy())
mem_dict['d'] = (0, test_case.copy())
mem_dict['e'] = (0, test_case.copy())


# storing results in case a 99 is thrown...think this is my issue
result_list = [0]

In [41]:
while True:
    if counter == 0:
        print("Counter is 0")
        i = 0
        for amp_type in mem_dict.keys():
            
            while result_list[-1] == None:
                
                # we can't pass a None value
                result_list.pop()
                
            output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], comb[i], result_list[-1], counter)
            
            # update memory 
            mem_dict[amp_type] = (ix, vals)
            
            # output
            result_list.append(output)
            
            i += 1
            
        # update counter 
        counter += 1
        
    else:
        i = 0
        
        for amp_type in mem_dict.keys():
            
            while result_list[-1] == None:
                
                # we can't pass a None value
                result_list.pop()
                
            output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], result_list[-1], result_list[-1], counter)
            
            # update memory 
            mem_dict[amp_type] = (ix, vals)
            
            # output
            result_list.append(output)
            
            i += 1
            
        # update counter 
        counter += 1     
    
    # halt process onces e_flag is True
    if amp_type == 'e' and result_list[-1] is None:

        # if e_output is greater than current final answer, overwrite
        if result_list[-2] > final_answer:
            final_answer = result_list[-2]

        print("Done")
        break

Counter is 0
Done


In [42]:
final_answer

139629729

### Try second test: 

- I believe I fell into the trap of a list being updated....needed to make a copy of it...(overview here...dumb...https://stackoverflow.com/questions/2612802/how-to-clone-or-copy-a-list)

In [43]:
# have a 0 start val for final answer
final_answer = 0

# iterate through just test that works - ensure output matches 
comb = (9,7,8,5,6)

counter = 0

test_case = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]

# start all at index 0
mem_dict['a'] = (0, test_case.copy())
mem_dict['b'] = (0, test_case.copy())
mem_dict['c'] = (0, test_case.copy())
mem_dict['d'] = (0, test_case.copy())
mem_dict['e'] = (0, test_case.copy())


# storing results in case a 99 is thrown...think this is my issue
result_list = [0]

In [44]:
while True:
    if counter == 0:
        print("Counter is 0")
        i = 0
        for amp_type in mem_dict.keys():
            
            while result_list[-1] == None:
                
                # we can't pass a None value
                result_list.pop()
                
            output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], comb[i], result_list[-1], counter)
            
            # update memory 
            mem_dict[amp_type] = (ix, vals)
            
            # output
            result_list.append(output)
            
            i += 1
            
        # update counter 
        counter += 1
        
    else:
        i = 0
        
        for amp_type in mem_dict.keys():
            
            while result_list[-1] == None:
                
                # we can't pass a None value
                result_list.pop()
                
            output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], result_list[-1], result_list[-1], counter)
            
            # update memory 
            mem_dict[amp_type] = (ix, vals)
            
            # output
            result_list.append(output)
            
            i += 1
            
        # update counter 
        counter += 1     
    
    # halt process onces e_flag is True
    if amp_type == 'e' and result_list[-1] is None:

        # if e_output is greater than current final answer, overwrite
        if result_list[-2] > final_answer:
            final_answer = result_list[-2]

        break

Counter is 0
Done


In [45]:
print(final_answer)

18216


### Now do our little permutation fun: 

In [51]:
# read in data & go!
# empty list
data_list = []

# need to ensure a clean run each time 
with open('day7_data.txt') as f:
    for line in f:
        data_list.append(line.split(','))

# convert to ints
test_case = [int(x) for x in data_list[0]]

In [52]:
# have a 0 start val for final answer
final_answer = 0

all_combinations = list(itertools.permutations([5,6,7,8,9]))

# iterate through just test that works - ensure output matches 
for comb in all_combinations:

    counter = 0

    # start all at index 0
    mem_dict['a'] = (0, test_case.copy())
    mem_dict['b'] = (0, test_case.copy())
    mem_dict['c'] = (0, test_case.copy())
    mem_dict['d'] = (0, test_case.copy())
    mem_dict['e'] = (0, test_case.copy())


    # storing results in case a 99 is thrown...think this is my issue
    result_list = [0]
    
    while True:
        if counter == 0:
            i = 0
            for amp_type in mem_dict.keys():

                while result_list[-1] == None:

                    # we can't pass a None value
                    result_list.pop()

                output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], comb[i], result_list[-1], counter)

                # update memory 
                mem_dict[amp_type] = (ix, vals)

                # output
                result_list.append(output)

                i += 1

            # update counter 
            counter += 1

        else:
            i = 0

            for amp_type in mem_dict.keys():

                while result_list[-1] == None:

                    # we can't pass a None value
                    result_list.pop()

                output, ix, vals = intcode_pc(mem_dict[amp_type][1], mem_dict[amp_type][0], result_list[-1], result_list[-1], counter)

                # update memory 
                mem_dict[amp_type] = (ix, vals)

                # output
                result_list.append(output)

                i += 1

            # update counter 
            counter += 1     

        # halt process onces e_flag is True
        if amp_type == 'e' and result_list[-1] is None:

            # if e_output is greater than current final answer, overwrite
            if result_list[-2] > final_answer:
                final_answer = result_list[-2]

            break

In [53]:
print(final_answer)

30872528
